# Econ 5315 Development Economics, Assignment 1

                                                                                               42135 Yongjai Lee
## Problem 1: Property Rights and Labor Market Outcomes

### Part 1: Research design and Summary statistics

1. The main research question of the paper is: what is the labor market effects from entitling property rights when the Peruvian government issued property titles to over 1.2 million urban households between 1996 and 2003.

2. There are two groups: a treatment group and a control group (untreated). The survey was conducted midway through the program. Treatment group is the neighborhoods in which property titling effort had begun at the time of the survey. The control group is the neighborhoods in which property titling effort had not begun at the time of the survey. The strategy employed by the author is an intention to treat (ITT) analysis. 

3. The 10 most important variables are the important covariates. sexhd, agehd, babies24, elemhd, segschlprog, hadporg, lotsize, plumbing, inherit, and invader. These are the variables that most likely explain the differences between squatters and nonsquatters.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_ind
from statsmodels.iolib.summary2 import summary_col
from linearmodels.iv import IV2SLS

In [2]:
df1 = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Entitled_to_work.dta')
df1.head()

,agehd,avgage,babies24,chtensec,collhd,commuter,elemhd,enter,enteryr,enteryr_fut,...,walk8,walk10,wrkage,wrkage2,wrkageent,wrkageent2,wrkageentsquat,wrkageentsquat2,wrkagesquat,wrkagesquat2
0,59,46.750000,0,Yes,0,0.0,0,0.0,0.0,3.0,...,1.0,1.0,3,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30,16.166666,0,Yes,0,0.0,0,0.0,0.0,3.0,...,1.0,1.0,4,16.0,0.0,0.0,0.0,0.0,0.0,0.0
2,43,18.750000,0,No,0,0.0,1,0.0,0.0,3.0,...,1.0,1.0,5,25.0,0.0,0.0,0.0,0.0,5.0,25.0
3,45,24.833334,0,Yes,0,0.0,0,0.0,0.0,3.0,...,1.0,1.0,5,25.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60,27.900000,0,Yes,0,0.0,0,0.0,0.0,3.0,...,1.0,0.0,8,64.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Consider only late neighborhoods
df1_for3 = df1.loc[df1['enter'] == 0]

# Squatters
df1_for3_sq = df1_for3.loc[df1_for3['squat'] == 1]
df1_for3_sq.describe()
df1_for3_sq[['sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']].describe()

,sexhd,agehd,babies24,elemhd,segschlprog,hadporg,lotsize,plumbing,inherit,invader
count,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000,321.000000
mean,0.239875,48.258567,0.137072,0.376947,0.925234,0.221184,182.651090,0.672897,0.146417,0.205607
std,0.427674,12.205985,0.370679,0.485378,0.263425,0.415693,221.760533,0.469888,0.354076,0.404776
min,0.000000,22.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000
25%,0.000000,39.000000,0.000000,0.000000,1.000000,0.000000,105.000000,0.000000,0.000000,0.000000
50%,0.000000,47.000000,0.000000,0.000000,1.000000,0.000000,143.000000,1.000000,0.000000,0.000000
75%,0.000000,56.000000,0.000000,1.000000,1.000000,0.000000,200.000000,1.000000,0.000000,0.000000
max,1.000000,88.000000,2.000000,1.000000,1.000000,1.000000,2000.000000,1.000000,1.000000,1.000000


In [4]:
# Nonsquatters
df1_for3_nsq = df1_for3.loc[df1_for3['squat'] == 0]
df1_for3_nsq.describe()
df1_for3_nsq[['sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']].describe()

,sexhd,agehd,babies24,elemhd,segschlprog,hadporg,lotsize,plumbing,inherit,invader
count,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1314.000000,1312.000000,1314.000000,1314.000000,1314.000000
mean,0.233638,51.392694,0.168189,0.416286,0.919330,0.187215,213.927591,0.837139,0.060122,0.213851
std,0.423305,12.579901,0.399762,0.493130,0.272431,0.390232,395.911090,0.369380,0.237803,0.410179
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,42.000000,0.000000,0.000000,1.000000,0.000000,108.000000,1.000000,0.000000,0.000000
50%,0.000000,51.000000,0.000000,0.000000,1.000000,0.000000,150.000000,1.000000,0.000000,0.000000
75%,0.000000,60.000000,0.000000,1.000000,1.000000,0.000000,200.000000,1.000000,0.000000,0.000000
max,1.000000,92.000000,3.000000,1.000000,1.000000,1.000000,9625.000000,1.000000,1.000000,1.000000


4. The group means are not significantly different. There is no reason for concern. It means that squatters and nonsquatters have no significance difference in their characteristics pre program.

In [5]:
# Show all
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ttest_ind(df1_for3_sq['sexhd'], df1_for3_nsq['sexhd'])
ttest_ind(df1_for3_sq['agehd'], df1_for3_nsq['agehd'])
ttest_ind(df1_for3_sq['babies24'], df1_for3_nsq['babies24'])
ttest_ind(df1_for3_sq['elemhd'], df1_for3_nsq['elemhd'])
ttest_ind(df1_for3_sq['segschlprog'], df1_for3_nsq['segschlprog'])
ttest_ind(df1_for3_sq['hadporg'], df1_for3_nsq['hadporg'])
ttest_ind(df1_for3_sq['lotsize'], df1_for3_nsq['lotsize'])
ttest_ind(df1_for3_sq['plumbing'], df1_for3_nsq['plumbing'])
ttest_ind(df1_for3_sq['inherit'], df1_for3_nsq['inherit'])
ttest_ind(df1_for3_sq['invader'], df1_for3_nsq['invader'])

# Last (default)
InteractiveShell.ast_node_interactivity = "last"

|               | mean (squatter) | Late neighborhoods (nonsquatter) |  pvalue for t-test    |
| ------------- |:-------------------:| :----------------: |:----:|
| sexhd    |        0.239875       | 0.233638              |0.813 |
| agehd  |          48.258567     |   51.392694           |5.96 |
|      babies24         |          0.137072       |      0.168189        |0.20 |
|      elemhd         |          0.376947       |      0.416286        |0.19 |
|      segschlprog         |          0.925234       |      0.919330        |0.72 |
|      hadporg         |          0.221184       |      0.187215        |0.16 |
|      lotsize         |          182.651090       |      213.927591        |NaN |
|      plumbing         |          0.672897       |      0.837139        |2.12 |
|      inherit         |          0.146417       |      0.060122        |1.83 |
|      invader         |          0.205607       |      0.213851        |0.74 |

### Part 2: Simple differences

1. Whether or not the household benefits from the program, being a squatter, have no significant impact on the total amount of hours worked in each household for early neighborhoods data (treatment group).

In [6]:
# Early neighborhoods data
df1_early = df1.loc[df1['enter'] == 1]

df1_early['const'] = 1

reg1 = sm.OLS(endog=df1_early['totwkhrs'], exog=df1_early[['const', 'squat']], \
              missing='drop')

type(reg1)

results = reg1.fit(cov_type='cluster',
                   cov_kwds={'groups': df1_early['segment']},
                   use_t=True)

type(results)

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.406
Date:                Fri, 08 Apr 2022   Prob (F-statistic):              0.239
Time:                        03:27:24   Log-Likelihood:                -4926.4
No. Observations:                 864   AIC:                             9857.
Df Residuals:                     862   BIC:                             9866.
Df Model:                           1                                         
Covariance Type:              cluster                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        100.9238      3.456     29.207      0.0

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


2. We can interpret the results as we don't see a significant relationship between the property rights and the hours worked in each household reached by the program. We might be seeing this results because of other conflicting covariates. There might be inherent differences between neighborhoods that affect both the probability of being a squatter and total hours worked. (such as average age)

3. The estimate of the impact does change. squat is almost significant at 5% level. The impact may have been downward biased. Squatters tend to work more.

In [7]:
# 3. 

reg1_controls = sm.OLS(endog=df1_early['totwkhrs'], exog=df1_early[['const', 'squat', 'sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']], \
              missing='drop')

type(reg1_controls)

results_controls = reg1_controls.fit(#cov_type='cluster',
                   #cov_kwds={'groups': df1_early['segment']},
                   #use_t=True)
)
type(results_controls)

print(results_controls.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     4.218
Date:                Fri, 08 Apr 2022   Prob (F-statistic):           4.12e-06
Time:                        03:27:24   Log-Likelihood:                -4859.7
No. Observations:                 856   AIC:                             9743.
Df Residuals:                     844   BIC:                             9800.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          29.0890     20.233      1.438      

4. Whether or not the household benefits from the program, being a squatter, has a significant impact on the total amount of hours worked in each household for early neighborhoods data (control group). However, note that r squared is very low meaning that the independent variable is not explaining much in the variation of the dependent variable. The estimates could be biased because we did not add any control variable. There might be inherent differences between neighborhoods (covariates) that affect both the property right and labour participation.

In [8]:
# late neighborhoods data

df1_for3['const'] = 1

reg2 = sm.OLS(endog=df1_for3['totwkhrs'], exog=df1_for3[['const', 'squat']], \
              missing='drop')

type(reg2)

results_2 = reg2.fit(cov_type='cluster',
                   cov_kwds={'groups': df1_for3['segment']},
                   use_t=True)

type(results_2)

print(results_2.summary(), results.summary())



                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     5.966
Date:                Fri, 08 Apr 2022   Prob (F-statistic):             0.0155
Time:                        03:27:24   Log-Likelihood:                -9486.2
No. Observations:                1635   AIC:                         1.898e+04
Df Residuals:                    1633   BIC:                         1.899e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        106.5426      2.584     41.230      0.0

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Results in a table

dfoutput = summary_col([results,results_controls,results_2],stars=True)
print(dfoutput)


             totwkhrs I totwkhrs II totwkhrs III
------------------------------------------------
R-squared   0.0008      0.0398      0.0033      
            0.0019      0.0521      0.0039      
agehd                   0.5707***               
                        (0.2004)                
babies24                22.4849***              
                        (6.0376)                
const       100.9238*** 29.0890     106.5426*** 
            (3.4555)    (20.2333)   (2.5841)    
elemhd                  9.2202*                 
                        (5.3427)                
hadporg                 -6.1358                 
                        (6.6504)                
inherit                 3.2819                  
                        (12.7271)               
invader                 -5.8423                 
                        (5.5944)                
lotsize                 -0.0039                 
                        (0.0086)                
plumbing           

### Part 3: Differences-in-differences Comparing Means

|               | Early neighborhoods | Late neighborhoods |      |
| ------------- |:-------------------:| :----------------: |:----:|
| Squatters     |        107.84       | 93.88              |13.96 |
| Nonsquatters  |          100.92     |   106.54           |-5.62 |
|               |          6.92       |      -12.66        |19.58 |

In [10]:
df1_early_sq = df1_early.loc[df1_early['squat'] == 1]
df1_early_nsq = df1_early.loc[df1_early['squat'] == 0]

a = df1_early_sq["totwkhrs"].mean()
b = df1_for3_sq["totwkhrs"].mean()
c = df1_early_nsq["totwkhrs"].mean()
d = df1_for3_nsq["totwkhrs"].mean()

print(a)
print(b)
print(c)
print(d)
print(a-b)
print(c-d)
print(a-c)
print(b-d)
print((a-c)-(b-d))

107.8423076923077
93.88473520249221
100.92384105960265
106.54261796042618
13.95757248981549
-5.6187769008235335
6.918466632705048
-12.657882757933976
19.576349390639024


### Part 4: Difference-in-difference in a Regression Framework

1. \begin{array}{l}
\Delta L_{i j k}=\beta_{0}+\beta_{1}\left(\text { squatter }_{i j k}\right)+\beta_{2}\left(\text { program periods }_{j k}\right) \\
+\beta_{3}\left(\text { program periods }_{j k} \cdot \text { squatter }_{i j k}\right)+\beta_{4}\left(N_{i j k}\right) \\
+\beta_{5}\left(\Delta N_{i j k}\right)+\alpha_{1}^{\prime} X_{i j k}+\alpha_{2}^{\prime} \Delta X_{i j k}+\varphi\left(C_{k}\right) \\
+\gamma\left(C_{k} \cdot \text { squatter }_{i j k}\right)+\phi\left(C_{k} \cdot \text { program }_{j k}\right)+e_{i j k} .
\end{array}

The above diff-in-diff regression specification can be simplified to the following.

$$Y=\beta_{0}+\beta_{1} t+\beta_{2} a+\beta_{3}(t * a)$$

where y = the observed outcome variable (totwkhrs)
      
t = time period (enter)
      
a = the segment of interest (squat)
      
a*t  = the interaction variable (enter $\times$ squat)

In [11]:
df1['const'] = 1

reg3 = sm.OLS(endog=df1['totwkhrs'], exog=df1[['const', 'squat', 'enter', 'entsquat']], \
              missing='drop')

type(reg3)

results_3 = reg3.fit(cov_type='cluster',
                     cov_kwds={'groups': df1['segment']},
                     use_t=True)

type(results_3)

print(results_3.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.492
Date:                Fri, 08 Apr 2022   Prob (F-statistic):             0.0605
Time:                        03:27:24   Log-Likelihood:                -14418.
No. Observations:                2499   AIC:                         2.884e+04
Df Residuals:                    2495   BIC:                         2.887e+04
Df Model:                           3                                         
Covariance Type:              cluster                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        106.5426      2.583     41.253      0.0

2. The coefficient for entsquat is the treatment effect: increase in labour participation from the property titling program. We see a positive and significant treatment effect, which makes sense. The diff-in-diff estimate for the effect of the program is 17.67 and is significant. The estimate is similar in magnitude when compared to estimates from Part 2 and 3. From part 2, the diff-in-diff estimate is 6.92 - (-12.66) = 19.58. From part 3, it is 19.58.

In [12]:
reg4 = sm.OLS(endog=df1['totwkhrs'], exog=df1[['const', 'squat', 'enter', 'entsquat', 'sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']], \
              missing='drop')

type(reg4)

results_4 = reg4.fit(#cov_type='cluster',
                     #cov_kwds={'groups': df1['segment']},
                     #use_t=True)
)
type(results_4)

print(results_4.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     6.739
Date:                Fri, 08 Apr 2022   Prob (F-statistic):           7.16e-13
Time:                        03:27:24   Log-Likelihood:                -14323.
No. Observations:                2489   AIC:                         2.867e+04
Df Residuals:                    2475   BIC:                         2.876e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          55.4640      9.774      5.675      

3. The results show insignificant positive treatment effect. Number of working age memebers have highly significant and positive affect on total hours worked.

In [13]:
reg5 = sm.OLS(endog=df1['totwkhrs'], exog=df1[['const', 'squat', 'enter', 'entsquat', 'wrkage','wrkageent' ,'wrkageentsquat', 'wrkagesquat','sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']], \
              missing='drop')

type(reg5)

results_5 = reg5.fit(#cov_type='cluster',
                     #cov_kwds={'groups': df1['segment']},
                     #use_t=True)
)
type(results_5)

print(results_5.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     73.82
Date:                Fri, 08 Apr 2022   Prob (F-statistic):          1.44e-205
Time:                        03:27:24   Log-Likelihood:                -13856.
No. Observations:                2489   AIC:                         2.775e+04
Df Residuals:                    2471   BIC:                         2.785e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -9.3848      8.894     -1.

4. The rationale is to see if regression has a quadratic relationship in terms of working hours. Treatment effect is large, positive, and significant. Does not seem to follow quadratic form.

In [14]:
reg6 = sm.OLS(endog=df1['totwkhrs'], exog=df1[['const', 'squat', 'enter', 'entsquat', 'wrkage','wrkageent' ,'wrkageentsquat', 'wrkagesquat','wrkage2', 'wrkageent2' ,'wrkageentsquat2', 'wrkagesquat2','sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg', 'lotsize', 'plumbing', 'inherit', 'invader']], \
              missing='drop')

type(reg6)

results_6 = reg6.fit(#cov_type='cluster',
                     #cov_kwds={'groups': df1['segment']},
                     #use_t=True)
)
type(results_6)

print(results_6.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     60.80
Date:                Fri, 08 Apr 2022   Prob (F-statistic):          2.87e-205
Time:                        03:27:24   Log-Likelihood:                -13848.
No. Observations:                2489   AIC:                         2.774e+04
Df Residuals:                    2467   BIC:                         2.787e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -4.1944     11.493     

In [15]:
# Results in a table

dfoutput_2 = summary_col([results_3,results_4,results_5,results_6],stars=True)
print(dfoutput_2)


                 totwkhrs I totwkhrs II totwkhrs III totwkhrs IIII
------------------------------------------------------------------
R-squared       0.0022      0.0291      0.3322       0.3354       
                0.0034      0.0342      0.3368       0.3410       
agehd                       0.5104***   -0.0538      -0.0547      
                            (0.1307)    (0.1098)     (0.1097)     
babies24                    16.4465***  8.6631***    8.2972**     
                            (3.8857)    (3.2337)     (3.2287)     
const           106.5426*** 55.4640***  -9.3848      -4.1944      
                (2.5827)    (9.7738)    (8.8944)     (11.4935)    
elemhd                      11.4322***  6.5105**     6.2358**     
                            (3.3456)    (2.7811)     (2.7762)     
enter           -5.6188     -6.3844*    -4.5036      4.3726       
                (4.2675)    (3.8029)    (7.7853)     (14.5335)    
entsquat        19.5763**   17.6696**   3.5505       56.2527*

### Part 5: Instrumental Variable Analysis

1. Because ITT estimates an experiment based on the initial treatment assignment. Diff in Diff methodology is uses ITT approach with control group, treatment group (pre, post treatment), and treatment effect.

2. We can't estimate the impact of property rights on labor supply for people who already have properties but had they not properties. This is hypothetical and cannot be estimated, but is what we want to measure. IV would help as we use the relationship between the instrument and dependent variable instead of independent variable and dependent variable. There are two main assumptions for iv. 1) They cause variation in the treatment variables and (2) they do not have a direct effect on the outcome variable, only indirectly through the treatment.

3. 2SLS model.

First stage: $$\text { hastitle }_{i}=\delta_{0}+\delta_{1} \text { squat }_{i}+v_{i}$$

Second Stage: $$\operatorname{totwkhrs}_{i}=\beta_{0}+\beta_{1}  \widehat{hastitle} r_{i}+u_{i}$$

IV results is not significant but is positive which means property right does, though not significant, has a positive affect on labour supply even when looking at people with properties.

In [16]:
# Fit the first stage regression and print summary
results_fs = sm.OLS(df1['hastitle'],
                    df1[['const', 'squat']],
                    missing='drop').fit()
print(results_fs.summary())

                            OLS Regression Results                            
Dep. Variable:               hastitle   R-squared:                       0.618
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     4038.
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        03:27:25   Log-Likelihood:                 178.75
No. Observations:                2499   AIC:                            -353.5
Df Residuals:                    2497   BIC:                            -341.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0000      0.005    194.336      0.0

In [17]:
df1['predicted_totwkhrs'] = results_fs.predict()

results_ss = sm.OLS(df1['totwkhrs'],
                    df1[['const', 'hastitle', 'sexhd','agehd','babies24', 'elemhd', 'segschlprog', 'hadporg']]).fit()
print(results_ss.summary())

                            OLS Regression Results                            
Dep. Variable:               totwkhrs   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     11.26
Date:                Fri, 08 Apr 2022   Prob (F-statistic):           4.14e-14
Time:                        03:27:25   Log-Likelihood:                -14383.
No. Observations:                2499   AIC:                         2.878e+04
Df Residuals:                    2491   BIC:                         2.883e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          48.0801      9.578      5.020      

In [18]:
iv = IV2SLS(dependent=df1['totwkhrs'],
            exog=df1['const'],
            endog=df1['hastitle'],
            instruments=df1['squat']).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               totwkhrs   R-squared:                      0.0013
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0009
No. Observations:                2499   F-statistic:                    1.5956
Date:                Fri, Apr 08 2022   P-value (F-stat)                0.2065
Time:                        03:27:25   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          97.927     4.8219     20.309     0.0000      88.477      107.38
hastitle       6.8458     5.4196     1.2632     0.20

In [19]:
# Results in a table (1st stage, 2nd stage, iv is above)

dfoutput_3 = summary_col([results_fs,results_ss],stars=True)
print(dfoutput_3)


             hastitle    totwkhrs 
----------------------------------
R-squared   0.6177     0.0279     
            0.6179     0.0307     
agehd                  0.5234***  
                       (0.1273)   
babies24               16.7248*** 
                       (3.8583)   
const       1.0000***  48.0801*** 
            (0.0051)   (9.5775)   
elemhd                 11.4045*** 
                       (3.3277)   
hadporg                -0.7870    
                       (3.9719)   
hastitle               5.3726     
                       (4.2213)   
segschlprog            21.8077*** 
                       (6.5212)   
sexhd                  -12.6819***
                       (3.6273)   
squat       -0.6781***            
            (0.0107)              
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


### Part 6: Policy Relevance

1. Because different methodologies strengthen the results. Also the validity of the methodology depends on arguments of each assumptions. Policy maker can decide which methodology seems more legitimate based on those arguments. IV's results are also more generalizable. IV results can be generalized to people with properties while it is questionable to do that generalization for ITT.

2. Diff in Diff has the best results and seems to be specific to peru. Bettering property rights in urban Peru seems to be a good policy to increase Labor participation.

3. Insecure property rights put a constraint on people's investment decisions. For example, there is low level of investment in developing countries government bonds because they default often. Another example might be low investment, especially in the East side of, in Ukraine.

## Problem 2: Smoke Inhalation and Low-Polluting Stoves

### Part 1: Research question and Summary statistics

1. The motivation is to find if improved cooking stoves can reduce indoor air pollution, improve health, and decrease greenhouse gas emissions in developing countries. This is a relevant research question because health, environmental aspects are important and if we can bring well-being by changing a small aspect of life that would be nice.

2. There are three assumptions: standardized interventions, equal groups and equal trial environments.

standardized interventions: interventions can be defined precisely and monitored.

equal groups environments: The group of individuals that receive intervention is expected to represent the broader population.

equal trial environments: trial environments are expected to be unaffected by other factors between groups.

3. Obtained permission from 42 villages in Orissa, India, to participate in the study. 3 withrew and added 5. Total of 2,541 households in 44 villages participated in the study.

Randomly distributed stoves among households => collected data

No need to cluster since the stove distribution was random. There is nothing correlated with the distribution of stoves. (Nothing to be biased)

4. traditional stove: 99.2%, clean stove: 95.4%, improved stove: 0.892%, average meals cooked last week: 18.34, average meals cooked last week with traditional stove: 15.29. The program has lots of different data. It has potential for household cooking consumption pattern. 

In [20]:
df2_1 = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Up_in_smoke_1.dta')
df2_1.head()

,hhid,hhid_M,village,treatment,maxmeals,mealslastweek_n_all,mealslastweek_n_IMPall,mealslastweek_n_CLEANall,mealslastweek_n_TRADall,cleanstove_in_HH,...,lessthan_primary_HOHfemale,primary_school_HOHmale,primary_school_HOHfemale,secondary_more_HOHmale,secondary_more_HOHfemale,hhsize,HHconsump_percap,Scaste_tribe,electricity,HHexpend_capita99
0,1001,1001,1,1.0,14.0,14.0,0.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,260.535706,1.0,0.0,260.535706
1,1002,1002,1,1.0,14.0,14.0,0.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,582.222229,1.0,0.0,582.222229
2,1003,1003,1,0.0,14.0,14.0,0.0,0.0,14.0,0.0,...,1.0,0.0,0.0,0.0,0.0,8.0,308.541656,1.0,0.0,308.541656
3,1004,1004,1,1.0,14.0,14.0,0.0,0.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,347.642853,1.0,0.0,347.642853
4,1005,1005,1,1.0,14.0,14.0,0.0,0.0,14.0,0.0,...,1.0,0.0,0.0,0.0,0.0,11.0,353.515167,1.0,0.0,353.515167


In [21]:
sum(df2_1.type_trad_number == 0)

20

In [22]:
(2578-20)/2578

0.9922420480993018

In [23]:
sum(df2_1.type_trad_number > 0)/2578

0.9546159813809154

In [24]:
sum(df2_1.type_imprvd_number > 0)/2578

0.008921644685802949

In [25]:
df2_1['mealslastweek_n_all'].mean()

18.33521270751953

In [26]:
df2_1['mealslastweek_n_TRADall'].mean()

15.286865234375

5. They look very similar across groups. We can see that it's randomized.

In [27]:
df2_1_treatment = df2_1.loc[df2_1['treatment'] == 1]

df2_1_treatment.describe()
df2_1_treatment[['hhsize', 'HHexpend_capita99', 'electricity', 'tradit_in_HH', 'cleanstove_in_HH', 'attendschool_HOHmale', 'attendschool_HOHfemale', 'mealslastweek_n_TRADall', 'mealslastweek_n_CLEANall']].describe()


,hhsize,HHexpend_capita99,electricity,tradit_in_HH,cleanstove_in_HH,attendschool_HOHmale,attendschool_HOHfemale,mealslastweek_n_TRADall,mealslastweek_n_CLEANall
count,1582.000000,1565.000000,1581.000000,1585.000000,1585.000000,1309.000000,1495.000000,1585.000000,1585.000000
mean,6.677623,475.482666,0.464896,0.992429,0.309148,0.683728,0.323746,15.133123,1.505994
std,3.659753,298.713837,0.498924,0.086708,0.611610,0.465198,0.468059,9.588079,5.059779
min,1.000000,45.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,278.750031,0.000000,1.000000,0.000000,0.000000,0.000000,14.000000,0.000000
50%,6.000000,405.520813,0.000000,1.000000,0.000000,1.000000,0.000000,14.000000,0.000000
75%,8.000000,581.111023,1.000000,1.000000,0.000000,1.000000,1.000000,14.000000,0.000000
max,30.000000,2158.000000,1.000000,1.000000,4.000000,1.000000,1.000000,96.000000,56.000000


In [28]:
df2_1_control = df2_1.loc[df2_1['treatment'] == 0]

df2_1_control.describe()
df2_1_control[['hhsize', 'HHexpend_capita99', 'electricity', 'tradit_in_HH', 'cleanstove_in_HH', 'attendschool_HOHmale', 'attendschool_HOHfemale', 'mealslastweek_n_TRADall', 'mealslastweek_n_CLEANall']].describe()


,hhsize,HHexpend_capita99,electricity,tradit_in_HH,cleanstove_in_HH,attendschool_HOHmale,attendschool_HOHfemale,mealslastweek_n_TRADall,mealslastweek_n_CLEANall
count,887.000000,874.000000,888.000000,888.000000,889.000000,734.000000,828.000000,889.000000,889.000000
mean,6.409245,469.872620,0.470721,0.989865,0.293588,0.715259,0.310386,15.547807,1.230596
std,3.162000,294.674255,0.499423,0.100218,0.602520,0.451598,0.462932,10.414857,4.598962
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,279.823608,0.000000,1.000000,0.000000,0.000000,0.000000,14.000000,0.000000
50%,6.000000,400.702393,0.000000,1.000000,0.000000,1.000000,0.000000,14.000000,0.000000
75%,8.000000,556.545288,1.000000,1.000000,0.000000,1.000000,1.000000,14.000000,0.000000
max,33.000000,2159.583252,1.000000,1.000000,4.000000,1.000000,1.000000,96.000000,44.000000


6. The data seems very similar. The means are not different.

In [29]:
df2_2 = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Up_in_smoke_2.dta')
df2_2.head()

,hhid_M,resid_M,hhid,resid,village_M,treatment,Lottery1,Lottery2,primarycook,female_bl,...,wheeze,tightchest,efever,cough,cold,cold_or_cough,phlegm,headache,soreeyes,anyillness
0,1001.0,1,1001,1,1,1.0,0.0,1.0,0.0,0.0,...,No,No,No,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,1001.0,2,1001,2,1,1.0,0.0,1.0,1.0,1.0,...,No,No,No,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,1001.0,3,1001,3,1,1.0,0.0,1.0,0.0,0.0,...,No,No,Yes,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,1001.0,4,1001,4,1,1.0,0.0,1.0,0.0,1.0,...,No,No,No,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,1002.0,1,1002,1,1,1.0,0.0,1.0,0.0,0.0,...,No,No,No,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [30]:
df2_2_treatment = df2_2.loc[df2_2['treatment'] == 1]
df2_2_treatment_pr = df2_2_treatment.loc[df2_2_treatment['primarycook'] == 1]

df2_2_treatment_pr.describe()
df2_2_treatment_pr[['COad_BL', 'cold_or_cough', 'phlegm', 'headache', 'soreeyes']].describe()


,COad_BL,cold_or_cough,phlegm,headache,soreeyes
count,1317.000000,1613.000000,1613.000000,1613.000000,1613.000000
mean,7.696280,0.513329,0.125852,0.481711,0.277743
std,6.067162,0.499978,0.331786,0.499822,0.448029
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,1.000000,0.000000,0.000000,0.000000
75%,10.000000,1.000000,0.000000,1.000000,1.000000
max,46.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
df2_2_control = df2_2.loc[df2_2['treatment'] == 0]
df2_2_control_pr = df2_2_control.loc[df2_2_control['primarycook'] == 1]

df2_2_control_pr.describe()
df2_2_control_pr[['COad_BL', 'cold_or_cough', 'phlegm', 'headache', 'soreeyes']].describe()



,COad_BL,cold_or_cough,phlegm,headache,soreeyes
count,723.000000,891.000000,890.000000,891.000000,891.000000
mean,7.907331,0.529742,0.140449,0.505050,0.292929
std,6.592285,0.499395,0.347650,0.500257,0.455360
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,1.000000,0.000000,1.000000,0.000000
75%,10.000000,1.000000,0.000000,1.000000,1.000000
max,47.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
# Show all
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ttest_ind(df2_2_treatment_pr['COad_BL'], df2_2_control_pr['COad_BL'])
ttest_ind(df2_2_treatment_pr['cold_or_cough'], df2_2_control_pr['cold_or_cough'])
ttest_ind(df2_2_treatment_pr['phlegm'], df2_2_control_pr['phlegm'])
ttest_ind(df2_2_treatment_pr['headache'], df2_2_control_pr['headache'])
ttest_ind(df2_2_treatment_pr['soreeyes'], df2_2_control_pr['soreeyes'])

# They are basically same

Ttest_indResult(statistic=nan, pvalue=nan)

In [33]:
# Last (default)
InteractiveShell.ast_node_interactivity = "last"

### Part 2: Take-up of the Program

1. stovebuilt has significant and positive effect on treatment. About 75.5% of treated built GV stoves. goodcond has significant and negative effect on treatment. The stove was not kept in a good condition.

In [34]:
df2_3 = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Up_in_smoke_3.dta')
df2_3

,hhid_M,hhid,stovebuilt,anystove,goodcond,mealslowpol_good,village,v_mo_control1,v_mo_control2,v_mo_control3,...,v_mo_control469,v_mo_control470,v_mo_control471,v_mo_control472,v_mo_control473,v_mo_control474,v_mo_control475,v_mo_control476,v_mo_control477,treat
0,1025.0,1025,0.0,0.0,0.0,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,1021.0,1021,NaN,NaN,NaN,10.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,1013.0,1013,NaN,NaN,NaN,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,1027.0,1027,0.0,0.0,NaN,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,1016.0,1016,0.0,0.0,0.0,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28434,47002.0,47002,NaN,NaN,NaN,14.0,47,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
28435,47008.0,47008,0.0,0.0,NaN,NaN,47,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
28436,47005.0,47005,NaN,NaN,NaN,0.0,47,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
28437,47011.0,47011,NaN,NaN,NaN,14.0,47,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [35]:
df2_3['const'] = 1

reg2_1 = sm.OLS(endog=df2_3['treat'], exog=df2_3[['const', 'stovebuilt','anystove', 'goodcond']], missing='drop')

type(reg2_1)

results2_1 = reg2_1.fit()

type(results2_1)

print(results2_1.summary())

#Omitted mealslowpol_good because it makes the data zero array.

                            OLS Regression Results                            
Dep. Variable:                  treat   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.492
Method:                 Least Squares   F-statistic:                     3994.
Date:                Fri, 08 Apr 2022   Prob (F-statistic):               0.00
Time:                        03:27:34   Log-Likelihood:                -4574.4
No. Observations:               12366   AIC:                             9157.
Df Residuals:                   12362   BIC:                             9187.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1705      0.004     40.226      0.0

### Part 3: Effects on Smoke Inhalation

1. There is no significant impact of the program on the level of CO inhaled by primary cooks at 5% level. There is a significant negative impact of the program on the level of CO inhaled by primary cooks at 10% level.

In [36]:
df2_ad = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Up_in_smoke_adults.dta')
df2_ad.head()

,hhid_M,hhid,resid,village_M,village_bl,male_bl,primarycook,anystove,CO_adult,v_mo_control1,...,v_mo_control269,v_mo_control270,v_mo_control271,v_mo_control272,v_mo_control273,v_mo_control274,v_mo_control275,v_mo_control276,v_mo_control277,treat
0,1022.0,1022,2,1,1.0,0.0,0.0,0.0,9.0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,1003.0,1003,3,1,1.0,1.0,0.0,0.0,3.0,0,...,0,1,0,0,0,0,0,0,0,0.0
2,1004.0,1004,1,1,1.0,1.0,0.0,0.5,13.0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,1012.0,1012,1,1,1.0,0.0,0.0,0.0,4.0,0,...,0,1,0,0,0,0,0,0,0,0.0
4,1013.0,1013,8,1,NaN,NaN,NaN,0.0,8.0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [37]:
df2_ad_pr = df2_ad.loc[df2_ad['primarycook'] == 1]

df2_ad_pr['const'] = 1

reg2_ad_pr = sm.OLS(endog=df2_ad_pr['CO_adult'], exog=df2_ad_pr[['const', 'treat']], missing='drop')

type(reg2_ad_pr)

results2_ad_pr = reg2_ad_pr.fit()

type(results2_ad_pr)

print(results2_ad_pr.summary())

                            OLS Regression Results                            
Dep. Variable:               CO_adult   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.571
Date:                Fri, 08 Apr 2022   Prob (F-statistic):             0.0589
Time:                        03:27:39   Log-Likelihood:                -13608.
No. Observations:                4234   AIC:                         2.722e+04
Df Residuals:                    4232   BIC:                         2.723e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1160      0.123     57.736      0.0

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2. There is a significant negative impact of ownership of any low-polluting stove on the level of CO inhaled by primary cooks at 5% level.

In [38]:
iv_ad = IV2SLS(dependent=df2_ad_pr['CO_adult'],
            exog=df2_ad_pr['const'],
            endog=df2_ad_pr['anystove'],
            instruments=df2_ad_pr['treat']).fit(cov_type='unadjusted')

print(iv_ad.summary)

#skipped first stage and 2nd stage

                          IV-2SLS Estimation Summary                          
Dep. Variable:               CO_adult   R-squared:                      0.0006
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0004
No. Observations:                4117   F-statistic:                    4.0671
Date:                Fri, Apr 08 2022   P-value (F-stat)                0.0437
Time:                        03:27:41   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          7.4183     0.2416     30.711     0.0000      6.9449      7.8917
anystove      -1.0571     0.5242    -2.0167     0.04

/opt/anaconda3/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


3. No significant results for both OSL and 2SLS.

In [39]:
df2_ch = pd.read_stata('/Users/yongjai/Jupyter_Notebook/Python/SSE/Development/Up_in_smoke_children.dta')
df2_ch.head()

,hhid_M,hhid,resid,village_M,village_bl,male_bl,CO_children,anystove,v_mo_control1,v_mo_control2,...,v_mo_control220,v_mo_control221,v_mo_control222,v_mo_control223,v_mo_control224,v_mo_control225,v_mo_control226,v_mo_control227,v_mo_control228,treat
0,1017.0,1017,5,1,1.0,1.0,NaN,0.0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
1,1016.0,1016,3,1,1.0,0.0,NaN,1.0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,1008.0,1008,3,1,1.0,1.0,2.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,1014.0,1014,4,1,NaN,1.0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,1018.0,1018,5,1,1.0,0.0,NaN,1.0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [40]:
# No primary cook

df2_ch['const'] = 1

reg2_ch = sm.OLS(endog=df2_ch['CO_children'], exog=df2_ch[['const', 'treat']], missing='drop')

type(reg2_ch)

results2_ch = reg2_ch.fit()

type(results2_ch)

print(results2_ch.summary())

                            OLS Regression Results                            
Dep. Variable:            CO_children   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1384
Date:                Fri, 08 Apr 2022   Prob (F-statistic):              0.710
Time:                        03:27:44   Log-Likelihood:                -13508.
No. Observations:                4395   AIC:                         2.702e+04
Df Residuals:                    4393   BIC:                         2.703e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3963      0.108     50.184      0.0

In [41]:
iv_ch = IV2SLS(dependent=df2_ch['CO_children'],
            exog=df2_ch['const'],
            endog=df2_ch['anystove'],
            instruments=df2_ch['treat']).fit(cov_type='unadjusted')

print(iv_ch.summary)

#skipped first stage and 2nd stage

                          IV-2SLS Estimation Summary                          
Dep. Variable:            CO_children   R-squared:                      0.0006
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0004
No. Observations:                4262   F-statistic:                    0.1180
Date:                Fri, Apr 08 2022   P-value (F-stat)                0.7312
Time:                        03:27:45   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.3500     0.2123     25.197     0.0000      4.9339      5.7662
anystove       0.1651     0.4805     0.3435     0.73

/opt/anaconda3/lib/python3.7/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


### Part 4: Discussion, Policy Relevance and Suggestions

1. Yes. Improving home infrastructure can reduce indoor air pollution, improve health, and decrease greenhouse gas emissions in developing countries. But maintanence of these equipments are important.

2. The stoves are not maintained enough. The program could have supervised and monitored once a year to check the condition of the stoves.

3. If households are not willing to pay the upfront cost of acquiring an improved stove and do not seem concerned about the problem they seek to address, it is conceivable that they would not be willing to pay the small, but recurring, everyday costs that are associated with switching from their traditional technology and maintaining the improved stove. In simple, the people who don't have good stoves tend to also not maintain their stoves well. This correlates with health, environmental outcomes and biases the results.

4. Spillovers include total wood used, fuel costs, time spent cooking, repairs, and time spent on repairs. The spillovers are not concerning. Treatment households could conduct all the cooking for the control group since they own the improved stove.

5. RCT. Randomly distribute air purifiers at home to households in India. Collect home pollution data a few years afterwards. We can use OLS and regress treated on home pollution.